# 10-3 워드투벡터(Word2Vec)
1) CBOW(Continuous Bag of Words): 주변의 단어로 중간 단어 예측 <br>
2) Skip-Gram: 중간의 단어로 주변 단어 예측

# 영어 Word2Vec

In [1]:
import nltk
nltk.download('punkt')
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /home/yeon_28/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# 데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x7f0ebbeb3730>)

## 1. 전처리

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
target_text = etree.parse(targetXML)

# <content> </content> 사이의 내용 추출
parse_text = '\n'.join(target_text.xpath('//content/text()'))

# content 중간의 (Audio) (Laughter) 등의 배경음 제거
content_text = re.sub(r'\([^)]*\)','', parse_text)

In [5]:
# 문장 토큰화
sent_text = sent_tokenize(content_text)

normalized_text=[]
for string in sent_text:
    tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
    normalized_text.append(tokens)
    
# 단어 토큰화
result = [word_tokenize(sentence) for sentence in normalized_text]
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


## 2. Word2Vec 훈련시키기

In [8]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=4, sg=0)
'''
    word_size; 워드 벡터의 특징 값. 임베딩된 벡터의 차원.
    window: context window 크기
    min_count: 단어의 최소 빈도 수 제한
    workers: 학습을 위한 프로세스 수
    sg: 0은 CBOW, 1은 skip-gram
'''

In [9]:
model_result = model.wv.most_similar('man')
model_result

[('woman', 0.8453181982040405),
 ('guy', 0.8178772330284119),
 ('boy', 0.7553020715713501),
 ('lady', 0.7542213201522827),
 ('girl', 0.7417510747909546),
 ('gentleman', 0.7245469093322754),
 ('soldier', 0.7214753031730652),
 ('kid', 0.704687774181366),
 ('poet', 0.6613127589225769),
 ('rabbi', 0.6518429517745972)]

## 3. 모델 저장

In [ ]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('eng_w2v') # 저장
loaded_model = KeyedVectors.load_word2vec_format('eng_w2v') # 로드

# 한국어 Word2Vec

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt

In [11]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")

('ratings.txt', <http.client.HTTPMessage at 0x7f0e6a8a07c0>)

In [12]:
train_data = pd.read_table('ratings.txt')
train_data[:5]

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [13]:
train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

False


In [15]:
train_data['document'] = train_data['document'].str.replace("^ㄱ-ㅎㅏ-ㅣ가-힣]","")
train_data[:5]

<ipython-input-15-cf0d66d3d328>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace("^ㄱ-ㅎㅏ-ㅣ가-힣]","")


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [16]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
okt = Okt()
tokenized_data = []

for sentence in train_data['document']:
    temp_X = okt.morphs(sentence, stem=True) # 토큰화
    temp_X = [word for word in temp_X if not word in stopwords]
    tokenized_data.append(temp_X)

In [ ]:
# 리뷰 길이 분포 확인
print('리뷰의 최대 길이 :',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이 :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = tokenized_data, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
model.wv.most_similar("최민식")